# SQLAlchemy, Sqlite, and Dates

## Setup

In [1]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
engine = create_engine("sqlite:///../Resources/dow.sqlite", echo=False)

## Explore Database

In [3]:
inspector = inspect(engine)
inspector.get_table_names()

['dow']

In [4]:
# Get a list of column names and types
columns = inspector.get_columns('dow')
for c in columns:
    print(c['name'], c["type"])
# columns

id INTEGER
quarter INTEGER
stock TEXT
date TEXT
open_price FLOAT
high_price FLOAT
low_price FLOAT
close_price FLOAT
volume INTEGER
percent_change FLOAT


In [5]:
engine.execute('SELECT * FROM dow LIMIT 5').fetchall()

[(1, 1, 'AA', '2011-01-07', 15.82, 16.72, 15.78, 16.42, 239655616, None),
 (2, 1, 'AA', '2011-01-14', 16.71, 16.71, 15.64, 15.97, 242963398, None),
 (3, 1, 'AA', '2011-01-21', 16.19, 16.38, 15.6, 15.79, 138428495, None),
 (4, 1, 'AA', '2011-01-28', 15.87, 16.63, 15.82, 16.13, 151379173, None),
 (5, 1, 'AA', '2011-02-04', 16.18, 17.39, 16.18, 17.14, 154387761, None)]

## Reflect and query dates

In [8]:
# Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect=True)
Dow = Base.classes.dow

In [9]:
session = Session(engine)

How many dates do we have?

In [12]:
# Total dates
session.query(func.count(Dow.date)).all()

e =engine.execute('''select count(date) from dow''')

In [ ]:
e.

What is the earliest date?

In [18]:
# Earliest Date
session.query(Dow.date).order_by(Dow.date).first()

engine.execute('''select max(date) from dow''').first()

('2011-06-24',)

What is the latest date?

In [11]:
# Latest Date
session.query(Dow.date).order_by(Dow.date.desc()).first()

('2011-06-24')

Find all of the dates great than `2011-03-01`

In [22]:
# session.query(Dow.date).\
#     filter(Dow.date > '2011-03-01').
#     order_by(Dow.date).all()

                                            
q = engine.execute('''select * from dow where date >'2011-03-01' ''').fetchall()
q

[(9, 1, 'AA', '2011-03-04', 16.81, 16.94, 16.13, 16.58, 109493077, None),
 (10, 1, 'AA', '2011-03-11', 16.58, 16.75, 15.42, 16.03, 114332562, None),
 (11, 1, 'AA', '2011-03-18', 15.95, 16.33, 15.43, 16.11, 130374108, None),
 (12, 1, 'AA', '2011-03-25', 16.38, 17.24, 16.26, 17.09, 95550392, None),
 (21, 1, 'AXP', '2011-03-04', 43.73, 44.68, 42.75, 43.72, 38985037, None),
 (22, 1, 'AXP', '2011-03-11', 43.86, 45.54, 43.53, 44.28, 37613429, None),
 (23, 1, 'AXP', '2011-03-18', 43.86, 44.47, 42.19, 44.17, 41757526, None),
 (24, 1, 'AXP', '2011-03-25', 44.75, 45.61, 44.1, 45.59, 30798332, None),
 (33, 1, 'BA', '2011-03-04', 72.47, 72.9, 68.6, 71.8, 27527285, None),
 (34, 1, 'BA', '2011-03-11', 71.6, 72.5, 69.91, 71.64, 25556296, None),
 (35, 1, 'BA', '2011-03-18', 71.17, 71.23, 67.34, 69.1, 29746370, None),
 (36, 1, 'BA', '2011-03-25', 70.29, 73.5, 70.03, 73.34, 24423545, None),
 (45, 1, 'BAC', '2011-03-04', 14.27, 14.48, 13.81, 14.12, 699671790, None),
 (46, 1, 'BAC', '2011-03-11', 14.18, 1

### Important Note! Sqlite does not support a date column type, but SQLAlchemy will allow you to work with dates in the iso format. [sqlite dates](http://docs.sqlalchemy.org/en/latest/dialects/sqlite.html)

# Quick Review of DateTime

In [13]:
import datetime as dt

In [14]:
# Print a date object and a datetime object 
print(dt.date.today())
print(dt.date(2017, 1 ,31))

2019-03-24
2017-01-31


In [15]:
print(dt.datetime.utcnow())
print(dt.datetime(2017, 1, 31))

2019-03-25 05:37:22.490502
2017-01-31 00:00:00


Calculate a time delta

In [16]:
# date 1 week ago from today
week_ago = dt.date.today() - dt.timedelta(days=7)

Query for the Dow closing price 1 week before `2011-04-08` using the datetime library

In [17]:
# Query for the Dow closing price for `CSCO` 1 week before `2011-04-08` using the datetime library
query_date = dt.date(2011, 4, 8) - dt.timedelta(days=7)
print("Query Date: ", query_date)

Query Date:  2011-04-01


In [18]:
session.query(Dow.date, Dow.close_price).\
    filter(Dow.stock == 'CSCO').\
    filter(Dow.date == query_date).all()

[('2011-04-01', 17.04)]

In [19]:
# Parse out just the day from the datetime object
dt.date.today().strftime("%d")

'24'

Putting it all together

In [20]:
# Query for all dates matching the 
# following date string in the format `%d`
date_str = "14"
session.query(Dow.date).\
    filter(func.strftime("%d", Dow.date) == date_str).all()

[('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14')]

## Your Turn!

DataSet Citation: 

Brown, M. S., Pelosi, M. & Dirska, H. (2013). Dynamic-radius Species-conserving Genetic Algorithm for 
the Financial Forecasting of Dow Jones Index Stocks. Machine Learning and Data Mining in Pattern 
Recognition, 7988, 27-41.
